In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
import nltk

In [4]:
import pandas as pd

In [6]:
data=pd.read_csv('WELFake_Dataset.csv')

MemoryError: 

In [ ]:
display(data)

In [ ]:
data=data.rename(columns={'Unnamed: 0':'index_no','label':'class'})

In [ ]:
display(data)

In [ ]:
data=data.drop(['index_no','title'],axis=1)
display(data.columns)

In [ ]:
dt=data.drop(['subject','date','title'],axis=1)
display(dt.columns)

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
data.dropna(inplace = True)

In [ ]:
data['text']=data['text'].astype(str)

In [ ]:
display(data['text'])

In [ ]:
import nltk
nltk.download('punkt')

# Tokenization

In [ ]:
data['text']=data['text'].apply(word_tokenize)

In [ ]:
display(data['text'])

# Stemming

In [ ]:
from nltk.stem.snowball import SnowballStemmer
porter=SnowballStemmer("english")
def stem_it(text):
    return [porter.stem(word) for word in text]
data['text']=data['text'].apply(stem_it)
print(data.head(10))

# Stopword Removal

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('english'))

In [ ]:
def stop_it(text):
    dt=[word for word in text if len(word)>2]
    return dt
data['text']=data['text'].apply(stop_it)
print(data.head(10))    

In [ ]:
data.dropna(inplace = True)

# Splitting

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(data['text'],data['class'],test_size=0.3)
display(X_train.head())
print('\n')
X_train=X_train.apply(lambda x: ' '.join(x))
X_test=X_test.apply(lambda x: ' '.join(x))
print(X_train)
print("\n")

In [ ]:
print(Y_test.value_counts())

# Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
my_tfidf=TfidfVectorizer(max_df=0.6)
tfidf_train=my_tfidf.fit_transform(X_train)
tfidf_test=my_tfidf.transform(X_test)
print(tfidf_train)


# RANDOM FOREST ALGORITHM

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score as acs
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

rf_model = rf.fit(tfidf_train, Y_train)

y_pred = rf_model.predict(tfidf_test)

precision, recall, fscore, train_support = score(Y_test, y_pred, pos_label=1, average='binary')
print('Precision: {} / Recall: {} / F1-Score: {} / Accuracy: {}'.format(round(precision, 3), round(recall, 3), round(fscore,3), round(acs(Y_test,y_pred), 3)))

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
class_label = [0, 1]
df_cm = pd.DataFrame(cm, index=class_label,columns=class_label)
sns.heatmap(df_cm, annot=True, fmt='d')
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

#PIPELINE

In [ ]:
import sklearn

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
pipe = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('model', LinearSVC())])
model_svc = pipe.fit(tfidf_train, Y_train)
svc_pred = model_svc.predict(X_test)

print("Accuracy of Pipeline: {}%".format(round(accuracy_score(Y_test, svc_pred)*100,2)))
print("\nConfusion Matrix of pipeline:\n")
cm = confusion_matrix(Y_test, svc_pred)
class_label = [0, 1]
df_cm = pd.DataFrame(cm, index=class_label,columns=class_label)
sns.heatmap(df_cm, annot=True, fmt='d')
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()
print("\nClassification Report of pipeline:\n")
print(classification_report(Y_test, svc_pred))